# Data Staging
### Extract, Transform, Load (ETL)

In [ ]:
# GOOGLE COLAB SETUP
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install pandas
import pandas as pd



In [ ]:
# Read the csv file into a pandas dataframe
df= pd.read_csv('Cookie Company Financials.csv')

In [ ]:
df

In [ ]:
# Print a Series with the data type of each column
print(df.dtypes)

**Transform**

In [ ]:
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

df[:25]

In [ ]:
# Convert the 'Units Sold' column to an integer
df['Units Sold'] = df['Units Sold'].astype(int)

df[:25]

Removing $ asign and change data types

In [ ]:
# The type of the 'Revenue per cookie' column is object. Convert it to a float
# And remove the dollar sign
df['Revenue per cookie'] = df['Revenue per cookie'].str.replace('$','').astype('float')

In [ ]:
df['Cost per cookie'] = df['Cost per cookie'].str.replace('$','').astype('float')

In [ ]:
df[:25]

**Drop Unrequired columns**

In [ ]:
df = df.drop(columns = ['Month Name'])

In [ ]:
df

**Generate measures/facts**

In [ ]:
df['Revenue'] = df['Units Sold'] * df['Revenue per cookie']
df['Cost'] = df['Units Sold'] * df['Cost per cookie']
df['Profit'] = df['Revenue'] - df['Cost']

In [ ]:
df

In [ ]:
df = df.rename({'Month Number':'Month'}, axis='columns')

In [ ]:
df.head()

In [ ]:
# Print the unique values of the 'Country' column
unique_countries = df['Country'].unique()

print(unique_countries)

**Extract country population**

In [ ]:
df_country = pd.read_csv('Country Population.csv')

In [ ]:
# Filter the DataFrame to include only the rows for the unique countries
filtered_country_df = df_country[df_country['Country'].isin(unique_countries)]

In [ ]:
filtered_country_df

In [ ]:
# Reset the index of the filtered DataFrame
filtered_country_df = filtered_country_df.reset_index(drop=True)


In [ ]:
filtered_country_df

## Dataset Integration (Enrich Sales Data)

In [ ]:
result = pd.merge(df, df_country, how="left", on=["Country"]) # Merge the two DataFrames on the 'Country' column
result

**Check for null values after merge**

In [ ]:
result.isnull().sum()

In [ ]:
result.dtypes

**Extract covid data**

In [ ]:
df_covid = pd.read_csv('covid-data.csv')

In [ ]:
df_covid

In [ ]:
df_covid.columns

**Transform**

In [ ]:
# Select the desired columns
df_covid = df_covid[['location','total_cases','date']]

In [ ]:
print(df_covid.dtypes)

In [ ]:
df['Country'].unique()

In [ ]:
# Check the number of null values
df_covid['total_cases'].isnull().sum() 

In [ ]:
# Drop the null values
df_cases = df_covid[['location','total_cases','date']]
df_cases = df_cases.dropna()

In [ ]:
df_cases

In [ ]:
# to convert date to datetime
df_cases['date'] = pd.to_datetime(df_cases['date'])

In [ ]:
df_cases

In [ ]:
df_cases.reset_index(inplace=True) # reset index
df_cases = df_cases.drop(columns=['index']) # drop the old index column
df_cases

In [ ]:
df_cases['Month'] = df_cases['date'].dt.month # extract month
df_cases['Year'] = df_cases['date'].dt.year # extract year

In [ ]:
df_cases

In [ ]:
df_country['Country'].unique()

## Practical Class 2

#### Bucketizing (binning or discretization of continuous data)
###### Equal-Width Binning: Divide the range of the data into a specified number of equally sized intervals (bins). This is achieved by determining the range of the data and dividing it by the desired number of bins.

In [ ]:
# Example of equal-frequency binning using pandas.qcut()
bins = pd.qcut(df_cases['total_cases'], q=5)
bins

In [ ]:
test_df_cases = df_cases.copy() # Creating a copy to check the bin

In [ ]:
# Add the bins as a new column to the DataFrame
test_df_cases['Total Cases Bins'] = bins

In [ ]:
# Analyze the distribution of cases across the bins
bin_counts = test_df_cases['Total Cases Bins'].value_counts().sort_index()

In [ ]:
# Display the distribution (Confirming an equal-width binning on the 'Total Cases')
print("Distribution of COVID-19 cases across different severity levels:")
print(bin_counts)

In [ ]:
mask = (df_cases['date'].dt.day == 1)
print(df_cases.loc[mask])

In [ ]:
df_cases = df_cases.loc[mask]
df_cases

In [ ]:
df_cases = df_cases.drop(columns=['date'])
df_cases = df_cases.rename({'location': 'Country'},axis=1)
df_cases

In [ ]:
# Replace values in a column with the desired value
print(result['Year'].unique())
print(df_cases['Year'].unique())

In [ ]:
result['Year'] = result['Year'].replace(2019,2022).replace(2018,2021)
result.head()

In [ ]:
result['Year'].unique()

In [ ]:
print(df_cases['Year'].unique())

In [ ]:
new_result = pd.merge(result, df_cases, how="left", on=["Country","Month", "Year"])
new_result

In [ ]:
new_result.isnull().sum()

### Surrogate Key Generation

In [ ]:
new_result['Surrogate Keys'] = range(1,len(new_result)+1)

In [ ]:
df = new_result.reindex(columns=['Surrogate Keys'] + list([c for c in new_result.columns if c!= 'Surrogate Keys']))

In [ ]:
# Remove commas from 'Population' and convert to integer
df['Population'] = df['Population'].str.replace(',', '').astype(int)

In [ ]:
df

### Loading the data for destination data warehouse/database

In [ ]:
df.to_csv('Stagged_data.csv')

### Checklist for Data Transformation step
##### -> Create a surrogate key (This will help in uniquely identifying records when we create dimension tables)
##### -> Formatted and split the date into separate columns for day, month, and year to align with the Date Dimension requirements.
##### -> Remove commas from 'Population' and convert to integer
##### -> Converting date columns to datetime for Standardization, correct Formatting, etc

### Aggregation
###### In this phase, you would aggregate the transformed data as needed for the fact tables. For instance, you might need to summarize daily sales data into monthly figures or calculate total cases per month for the COVID data.

In [ ]:
# Aggregating COVID data by Country and Year, summing total cases and deaths
covid_aggregated = df.groupby(['Country', 'Year'])['total_cases'].sum().reset_index()
covid_aggregated

In [ ]:
# Aggregating Sales data by Country and Year, summing Units Sold, Revenue, and Cost
sales_aggregated = df.groupby(['Country', 'Year'])['Units Sold'].sum().reset_index()
sales_aggregated

In [ ]:
financials_aggregated = df.groupby(['Country', 'Date']).agg({
    'Revenue': 'sum'
}).reset_index()

In [ ]:
financials_aggregated